In [6]:
#Import libraries
from fastai.vision.all import *
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Optimizer 

In [ ]:
#We want to test different optimizers using different optimizer evaluation functions
#Run each optimizer across a) the Rosenbrock function, b) Local Minima filled functions, c) Bowl-Shaped functions, d) Plate-Shaped functions
# and e) Functions with Steep drops

#After running each optimizer across these functions, take the average across all functions and plot to determine average performance
#Generate  x data for each test - use same initialization distribution to ensure reproducibility and fairness while testing

#Function to calculate the loss (Euclidean distance between current point and global minima)
#The loss will be the DISTANCE from the local minima to the gradient - we're measuring how fast the optimizer can reach this minima
import math
def computeEuclideanLoss(current_point, global_minima):
    #Compute the squared difference between the two vectors
    squared_difference = torch.float_power(torch.subtract(current_point, global_minima), 2.0)
    #Add all squared differences, convert the resulting scalar to a float, and then take the square root
    return math.sqrt(torch.sum(squared_difference).item())

#Initialize x and y vectors - 3D functions will be written in the form z(x, y)
#Both x and y will be initialized with normal distributions from ranges -5 to 5
XYZ = torch.FloatTensor(3, 1).uniform_(-5, 5)

#Import Custom Adam Implementation
from CustomAdam import CustomAdam

#Fit the given 3D function
def fitFunction(epochs, xyz, minima, optimizer_function):
    losses = []
    #Iterate over number of epochs
    for epoch in epochs:
        #Set all gradients to zero - this is to prevent gradient accumulation as they do not reset per epoch
        Optimizer.zero_grad()
        #Compute the loss
        loss = computeEuclideanLoss(xyz, minima)
        #Compute the gradients
        loss.backward()
        #Run the optimizer
        optimizer_function.step()
        #Add loss to loss array
        losses.append(loss)
    #Return losses array
    return losses

In [ ]:
#Dictionaries of all functions to be fit (equations) and their minima
OPTIMIZER_TEST_FUNCTIONS = {"ROSENBROCK" : "(1 - {})**2 + 100*({}-({}**2))**2".format()}